In [1]:
#Define rooms for the dungeon escape

dungeon_room = {
    "name": "dungeon room",
    "type": "room",
}

torture_chamber = {
    "name": "torture chamber",
    "type": "room",
}

armory = {
    "name": "armory",
    "type": "room",
}

prison = {
    "name": "prison",
    "type": "room",
}

outside = {
    "name": "outside",
    "type": "room",
}

#Define items

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

door_b = {
    "name": "door b",
    "type": "door",
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

door_c = {
    "name": "door c",
    "type": "door",
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

door_d = {
    "name": "door d",
    "type": "door",
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

ladder = {
    "name": "ladder",
    "type": "furniture",
}

window = {
    "name": "window",
    "type": "furniture",
}

chains = {
    "name": "chains",
    "type": "furniture",
}

armor_stand = {
    "name": "armor stand",
    "type": "furniture",
}

table = {
    "name": "creaky table",
    "type": "furniture", #touching this will alert the guard
}

riddle_scroll = {
    "name": "riddle scroll",
    "type": "furniture",
}

#object relations

object_relations = {
    "dungeon room":[chains, door_a],
    "torture chamber":[door_a, door_b, armor_stand, riddle_scroll],
    "armory":[door_b, door_c, table],
    "prison":[window, ladder],
    "outside":[window],
    "door a":[dungeon_room, torture_chamber],
    "door b": [torture_chamber, armory],
    "door c": [armory],
    "table": ["guard_alert"], #Guard is alerted when this is touched
    "window": [prison, outside],
    "ladder": [prison, window],
    "chains": [key_a],
    "riddle_scroll": [key_b],
}

all_rooms = [dungeon_room, torture_chamber, armory, prison, outside]
all_doors = [door_a, door_b, door_c, door_d]

In [3]:

#define game state

INIT_GAME_STATE = {
    "current_room": dungeon_room,
    "keys_collected": [],
    "target_room": outside,
    "guard_alerted": False,
    "riddle_solved": False,
}


  # @title Default title text
def linebreak():
  print("\n\n")

def start_game():
  print("You find yourself imprisoned in a dark dungeon. You must escape before the guard catches you!")
  play_room(game_state["current_room"])

def play_room(room):
  game_state["current_room"] = room
  if game_state["guard_alerted"] and game_state ["current_room"] != prison:
    print ("The guard caught you. You are thrown into the prison")
    game_state ["current_room"] = prison
    print ("You need to find a way to escape")
    intended_action = input ("What would you like to do? Type 'explore' or 'examine'?").strip()
    if intended_action == "explore":
      explore_room (prison)
    elif intended_action == "examine":
      examine_item (input ("What would you like to examine?").strip())
      #if examine_item == ladder:
        #print ("You climb the ladder and escape through the window!")
        #game_state ["current_room"] = outside
        #play_room (outside)
    else:
      print ("Not sure what you mean. Type 'explore' or 'examine'.")
    play_room(prison)
    return

  elif game_state ["current_room"] == game_state["target_room"]:
    print("Congrats! You escaped the dungeon!")
    return
  else:
    print("You are now in " + room["name"])
    intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
    if intended_action == "explore":
      explore_room(room)
      play_room(room)
    elif intended_action == "examine":
      examine_item(input("What would you like to examine?").strip())
    else:
      print("Not sure what you mean. Type 'explore' or 'examine'.")
      play_room(room)
    linebreak()

def explore_room(room):
  items = [i["name"] for i in object_relations[room["name"]]]
  print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
  connected_rooms = object_relations[door["name"]]
  for room in connected_rooms:
    if(not current_room == room):
      return room

def solve_riddle ():   #Present a simple riddle to the player to solve
  print ("You found a scroll with a riddle on it: 'I am tall when I am young. I am short when I am old. What am I?'")
  answer = input("Your answer: ").strip().lower()
  if answer == "candle":
    print("Correct! You find a key for door b hidden behind the scroll")
    game_state ["riddle_solved"] = True
    key_found = key_b
    game_state ["keys_collected"].append(key_found)

  else:
    print("wrong answer. Try again later.")
    examine_item ("riddle scroll")

  play_room(game_state["current_room"])

def examine_item(item_name):
  current_room = game_state["current_room"]
  next_room = ""
  output = ""

  for item in object_relations[current_room["name"]]:
    if item["name"] == item_name:
      output = "You examine " + item_name + ". "
      if item["type"] == "door":
        have_key = False
        for key in game_state["keys_collected"]:
          if key["target"] == item:
            have_key = True
            break
        if(have_key):
          output += "You unlock it with a key you have."
          next_room = get_next_room_of_door(item, current_room)
        else:
          output += "It is locked but you don't have the key."
      elif item_name == "creaky table":
        output += "The table creaks loudly! The guard is alerted!"
        game_state ["guard_alerted"] = True
        print (output)
        play_room (prison)
        return
      elif item_name == "riddle scroll" and not game_state ["riddle_solved"]:
        solve_riddle()
        return
      elif item ["name"] == "ladder":
        print ("You climb the ladder to the window and escape through it!")
        game_state ["current_room"] = outside
        print ("Congrats! You escaped the dungeon!")
        return
      elif item ["type"] == "furniture" and item["name"] in object_relations:
        if len (object_relations [item["name"]]) > 0:
          item_found = object_relations[item["name"]].pop()
          game_state ["keys_collected"].append(item_found)
          output += "You find" + item_found ["name"] + "."
        else:
          output += "There isn't anything interesting about it"

      else:
        output += "There isn't anything iteresting about it"
      print(output)
      break

  if output == "":
    print("The item you requested is not found in the current room.")

  if next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'").strip() == 'yes':
    play_room (next_room)
  else:
    play_room(current_room)

In [5]:
game_state = INIT_GAME_STATE.copy()

start_game()

You find yourself imprisoned in a dark dungeon. You must escape before the guard catches you!
You are now in dungeon room


What would you like to do? Type 'explore' or 'examine'? explore


You explore the room. This is dungeon room. You find chains, door a
You are now in dungeon room


What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? chains


You examine chains. You findkey for door a.
You are now in dungeon room


...

You examine creaky table. The table creaks loudly! The guard is alerted!
You are now in prison


What would you like to do? Type 'explore' or 'examine'? explore


You explore the room. This is prison. You find window, ladder
You are now in prison


What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? ladder


You climb the ladder to the window and escape through it!
Congrats! You escaped the dungeon!




































